In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.metrics import classification_report, accuracy_score

tf.__version__

'2.4.1'

In [2]:
# -------- TEST USER ----------- #

TEST_USER      = '007'

BASE_DIR       = '../'
IMG_DIR        = 'RGB-Spatial-Path-Images/'
LOG_DIR        = 'Logs/'

USERS          = ['001', '002', '003', '004', '005', '006', '007']

# ------------------------------- Only Dynalic Gestures ------------------------------ #
GESTURES       = ['j', 'z', 'bad', 'deaf', 'fine', 'good', 'goodbye', 'hello', 'hungry',
                  'me', 'no', 'please', 'sorry', 'thankyou', 'yes', 'you']

PLANES         = ['XY', 'YZ', 'ZX']

BATCH_SIZE     = 32
IMG_LEN        = 160
IMG_SIZE       = (IMG_LEN, IMG_LEN)

# ------------- FOR THE GREATER GOOD :) ------------- #
TRAIN_LEN      = 960
TEST_LEN       = 160

EPOCHS         = 7
LEARNING_RATE  = 0.001

In [3]:
def load_data(plane):
    X_train = np.zeros((TRAIN_LEN, IMG_LEN, IMG_LEN, 3))
    X_test = np.zeros((TEST_LEN, IMG_LEN, IMG_LEN, 3))
    y_train = np.zeros((TRAIN_LEN, 1))
    y_test = np.zeros((TEST_LEN, 1))
    
    train_count = 0
    test_count = 0
        
    for gesture in GESTURES:
        print('loading data for ' + gesture + ' gesture on the ' + plane + ' plane ... ', end='')
        path = os.path.join(BASE_DIR, IMG_DIR, plane, gesture)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename))
            resized = cv2.resize(img, IMG_SIZE)
            if filename[1:4] != TEST_USER:
                X_train[train_count, :] = resized
                y_train[train_count, 0] = GESTURES.index(gesture)
                train_count = train_count + 1
            else:
                X_test[test_count, :] = resized
                y_test[test_count, 0] = GESTURES.index(gesture)
                test_count = test_count + 1
                
        print('√')
        
    return X_train, X_test, y_train, y_test

In [4]:
X_train_xy, X_test_xy, y_train_xy, y_test_xy = load_data('XY')
X_train_yz, X_test_yz, y_train_yz, y_test_yz = load_data('YZ')
X_train_zx, X_test_zx, y_train_zx, y_test_zx = load_data('ZX')

loading data for j gesture on the XY plane ... √
loading data for z gesture on the XY plane ... √
loading data for bad gesture on the XY plane ... √
loading data for deaf gesture on the XY plane ... √
loading data for fine gesture on the XY plane ... √
loading data for good gesture on the XY plane ... √
loading data for goodbye gesture on the XY plane ... √
loading data for hello gesture on the XY plane ... √
loading data for hungry gesture on the XY plane ... √
loading data for me gesture on the XY plane ... √
loading data for no gesture on the XY plane ... √
loading data for please gesture on the XY plane ... √
loading data for sorry gesture on the XY plane ... √
loading data for thankyou gesture on the XY plane ... √
loading data for yes gesture on the XY plane ... √
loading data for you gesture on the XY plane ... √
loading data for j gesture on the YZ plane ... √
loading data for z gesture on the YZ plane ... √
loading data for bad gesture on the YZ plane ... √
loading data for de

In [5]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [6]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.trainable = False

In [7]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(GESTURES))

In [8]:
def get_model():
    inputs = tf.keras.Input(shape=IMG_SHAPE)
    x = preprocess_input(inputs)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [9]:
model_xy = get_model()
history_xy = model_xy.fit(X_train_xy, y_train_xy, epochs=EPOCHS)

Epoch 1/7
30/30 [==============================] - 12s 338ms/step - loss: 2.7778 - accuracy: 0.1681
Epoch 2/7
30/30 [==============================] - 10s 331ms/step - loss: 1.5839 - accuracy: 0.5410
Epoch 3/7
30/30 [==============================] - 10s 330ms/step - loss: 1.1985 - accuracy: 0.6515
Epoch 4/7
30/30 [==============================] - 10s 332ms/step - loss: 0.9022 - accuracy: 0.7616
Epoch 5/7
30/30 [==============================] - 10s 332ms/step - loss: 0.7353 - accuracy: 0.8121
Epoch 6/7
30/30 [==============================] - 10s 329ms/step - loss: 0.6457 - accuracy: 0.8323
Epoch 7/7
30/30 [==============================] - 10s 337ms/step - loss: 0.5789 - accuracy: 0.8527


In [10]:
prob_xy = tf.keras.Sequential([model_xy, tf.keras.layers.Softmax()])
y_pred_xy = prob_xy.predict(X_test_xy)
y_pred = np.argmax(y_pred_xy, axis=1)
print(classification_report(y_test_xy.ravel(), y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        10
         1.0       0.43      1.00      0.61        10
         2.0       0.40      0.20      0.27        10
         3.0       0.00      0.00      0.00        10
         4.0       0.75      0.30      0.43        10
         5.0       0.67      0.40      0.50        10
         6.0       0.39      0.90      0.55        10
         7.0       0.58      0.70      0.64        10
         8.0       0.91      1.00      0.95        10
         9.0       0.00      0.00      0.00        10
        10.0       0.50      0.20      0.29        10
        11.0       0.67      0.20      0.31        10
        12.0       0.40      0.20      0.27        10
        13.0       0.88      0.70      0.78        10
        14.0       1.00      1.00      1.00        10
        15.0       0.10      0.30      0.15        10

    accuracy                           0.51       160
   macro avg       0.54   

/home/andromeda/Ai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/andromeda/Ai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/andromeda/Ai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
model_yz = get_model()
history_yz = model_yz.fit(X_train_yz, y_train_yz, epochs=EPOCHS)

Epoch 1/7
30/30 [==============================] - 12s 335ms/step - loss: 2.9388 - accuracy: 0.2077
Epoch 2/7
30/30 [==============================] - 10s 335ms/step - loss: 1.7374 - accuracy: 0.4113
Epoch 3/7
30/30 [==============================] - 10s 332ms/step - loss: 1.0617 - accuracy: 0.6472
Epoch 4/7
30/30 [==============================] - 10s 338ms/step - loss: 0.7966 - accuracy: 0.7575
Epoch 5/7
30/30 [==============================] - 10s 336ms/step - loss: 0.7452 - accuracy: 0.7713
Epoch 6/7
30/30 [==============================] - 10s 332ms/step - loss: 0.5469 - accuracy: 0.8558
Epoch 7/7
30/30 [==============================] - 10s 331ms/step - loss: 0.4850 - accuracy: 0.8746


In [12]:
prob_yz = tf.keras.Sequential([model_yz, tf.keras.layers.Softmax()])
y_pred_yz = prob_yz.predict(X_test_yz)
y_pred = np.argmax(y_pred_yz, axis=1)
print(classification_report(y_test_yz.ravel(), y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        10
         1.0       0.45      1.00      0.62        10
         2.0       0.50      0.80      0.62        10
         3.0       0.00      0.00      0.00        10
         4.0       1.00      0.20      0.33        10
         5.0       1.00      0.30      0.46        10
         6.0       0.82      0.90      0.86        10
         7.0       0.71      0.50      0.59        10
         8.0       0.91      1.00      0.95        10
         9.0       0.88      0.70      0.78        10
        10.0       0.77      1.00      0.87        10
        11.0       0.75      0.90      0.82        10
        12.0       0.78      0.70      0.74        10
        13.0       0.62      1.00      0.77        10
        14.0       0.67      0.80      0.73        10
        15.0       0.75      0.60      0.67        10

    accuracy                           0.71       160
   macro avg       0.73   

/home/andromeda/Ai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/andromeda/Ai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/andromeda/Ai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
model_zx = get_model()
history_zx = model_zx.fit(X_train_zx, y_train_zx, epochs=EPOCHS)

Epoch 1/7
30/30 [==============================] - 12s 337ms/step - loss: 3.3891 - accuracy: 0.1929
Epoch 2/7
30/30 [==============================] - 10s 343ms/step - loss: 2.3822 - accuracy: 0.3031
Epoch 3/7
30/30 [==============================] - 10s 336ms/step - loss: 1.6843 - accuracy: 0.4457
Epoch 4/7
30/30 [==============================] - 10s 331ms/step - loss: 1.2451 - accuracy: 0.5971
Epoch 5/7
30/30 [==============================] - 10s 328ms/step - loss: 1.1279 - accuracy: 0.6557
Epoch 6/7
30/30 [==============================] - 10s 331ms/step - loss: 0.8854 - accuracy: 0.7133
Epoch 7/7
30/30 [==============================] - 10s 326ms/step - loss: 0.8496 - accuracy: 0.7595


In [14]:
prob_zx = tf.keras.Sequential([model_zx, tf.keras.layers.Softmax()])
y_pred_zx = prob_zx.predict(X_test_zx)
y_pred = np.argmax(y_pred_zx, axis=1)
print(classification_report(y_test_zx.ravel(), y_pred))

              precision    recall  f1-score   support

         0.0       0.54      0.70      0.61        10
         1.0       0.90      0.90      0.90        10
         2.0       0.60      0.60      0.60        10
         3.0       0.29      0.90      0.44        10
         4.0       0.00      0.00      0.00        10
         5.0       0.78      0.70      0.74        10
         6.0       0.77      1.00      0.87        10
         7.0       0.53      0.80      0.64        10
         8.0       0.77      1.00      0.87        10
         9.0       0.00      0.00      0.00        10
        10.0       1.00      0.50      0.67        10
        11.0       0.00      0.00      0.00        10
        12.0       0.00      0.00      0.00        10
        13.0       1.00      0.90      0.95        10
        14.0       1.00      0.90      0.95        10
        15.0       0.25      0.10      0.14        10

    accuracy                           0.56       160
   macro avg       0.53   

/home/andromeda/Ai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/andromeda/Ai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/andromeda/Ai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
y_total = y_pred_xy * y_pred_yz * y_pred_zx
y_pred = np.argmax(y_total, axis=1)
report = classification_report(y_test_xy.ravel(), y_pred, zero_division=0)
print(report)

              precision    recall  f1-score   support

         0.0       0.71      1.00      0.83        10
         1.0       0.50      1.00      0.67        10
         2.0       0.64      0.90      0.75        10
         3.0       0.33      0.10      0.15        10
         4.0       0.00      0.00      0.00        10
         5.0       1.00      0.80      0.89        10
         6.0       0.83      1.00      0.91        10
         7.0       0.83      1.00      0.91        10
         8.0       1.00      1.00      1.00        10
         9.0       0.80      0.40      0.53        10
        10.0       1.00      1.00      1.00        10
        11.0       0.00      0.00      0.00        10
        12.0       0.39      0.70      0.50        10
        13.0       1.00      1.00      1.00        10
        14.0       0.91      1.00      0.95        10
        15.0       0.80      0.80      0.80        10

    accuracy                           0.73       160
   macro avg       0.67   

In [16]:
config = '\n\nTEST_USER ' + TEST_USER + '\n'
underline = '=====================================\n'
log_dir = os.path.join(BASE_DIR, LOG_DIR)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)
f = open(os.path.join(log_dir, 'logs_tl_rgb.txt'), 'a')
f.write(config)
f.write(underline)
f.write(report)
f.close()

In [ ]:
'''

            Spatial Path TL RGB
        ----------------------------
           USER           ACCURACY
           =======================
            001              86%
            002              82%
            003              93%
            004              53%
            005              76%
            006              82%
            007              73%

'''